In [1]:
from datasets import load_dataset
hotpot_qa = load_dataset("hotpot_qa","distractor")

/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hp = hotpot_qa['train'][0]

In [4]:
shuffled_dataset = hotpot_qa['train'].shuffle(42)

In [5]:
subset_data = shuffled_dataset[0:5000]

In [28]:
titles = [sd['title'] for sd in subset_data['context']]

In [6]:
titles = []
for sd in subset_data['context']:
    titles.extend(sd['title'])

In [7]:
from langchain_community.document_loaders import WikipediaLoader
import wikipediaapi

wikipedia_module = wikipediaapi.Wikipedia("HyDE-Project (athecolab@gmail.com)", "en")
# page_py = wikipedia_module.page(titles[1])

In [9]:
import backoff
import requests
@backoff.on_exception(backoff.expo,requests.exceptions.Timeout,max_tries=5,max_time=30)
def get_wikipedia(title):
    page_py = wikipedia_module.page(title)
    title_text = ""
    for st in page_py.sections:
        if st.title.lower() == 'references' or st.title.lower() == 'external links': continue
        title_text += st.text + "\n"
    return title_text

from tqdm import tqdm
import os
import pandas as pd

def is_file_empty(file_path):
    return os.path.getsize(file_path) == 0

if not os.path.exists(f"wikipedia_data.csv"):
    df = pd.DataFrame(columns=["title","text"],index=None)
    df.to_csv(f"wikipedia_data.csv",index=False,header=True)
else:
    df = pd.read_csv(f"wikipedia_data.csv",index_col=False)

if not os.path.exists(f"done_wikipedia.txt"):
    os.mknod(f"done_wikipedia.txt")

for title in tqdm(titles):
    # docs = WikipediaLoader(
    #         query=title, load_max_docs=1, doc_content_chars_max=1
    #     ).load()
    # wikipedia_title = docs[0].metadata["title"]
    if not is_file_empty(f"done_wikipedia.txt"):
        with open(f"done_wikipedia.txt","r") as f:
            done_titles = [x for x in f.read().splitlines()]
        # print(done_pids)
        if title in done_titles: 
            continue
    title_text = get_wikipedia(title)
    df_dict = {
        "title":[title],
        "text":[title_text]
    }
    df = pd.DataFrame(df_dict)
    
    df.to_csv(f"wikipedia_data.csv", mode='a',index=False,header=False)
    with open(f"done_wikipedia.txt", "a") as f:
        f.write(f"{title}\n")

100%|██████████| 49772/49772 [1:16:24<00:00, 10.86it/s]  
